### Import knihoven a konfigurace

In [ ]:
import argparse
import os
import shutil
import re
import math

import numpy as np
import matplotlib.pyplot as plt
import cv2
import yaml

from IPython.display import Image, display

from scipy.spatial import distance
from natsort import natsorted

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

### Pomocné funkce


In [ ]:
def show_images(*imgs, window_name='Image preview'):
    """ Opens multiple image previews depending on the length of the input *imgs list.
    The preview is terminated by pressing the 'q' key.
    
    Parameters
    ----------
    *imgs : list
        Multiple input images which have to be shown.
    window_name : Optional[string]
        An optional window name.
    Returns
    -------
    None
    """
    for i, img in enumerate(imgs, 1):
        window_name_id = window_name + ' ' + str(i)
        cv2.namedWindow(window_name_id, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
        cv2.resizeWindow(window_name_id, 1080, 720)
        cv2.moveWindow(window_name_id, 0, 0)

    while 1:
        for i, img in enumerate(imgs, 1):
            cv2.imshow(window_name + ' ' + str(i), img)
            
        k = cv2.waitKey(1) & 0xFF

        if k == ord('q'):
            break

    cv2.destroyAllWindows()

### Úkol 1

Úkol je zaměřen na segmentace obrazů, získání kontur objektů a měření poměru rozměrů v cm a pixelech.

**1) Načtěte a zkontrolujte zaznamenaný obrázek vzoru pomocí kamery. Nadefinujte proměné s šířkou a výškou referečního objektu.**

In [ ]:
img_patterns = ###
show_images(img_patterns)

In [ ]:
ref_width_real = ###
ref_height_real = ###

**2) Využijte funkci [cv2.inRange()](https://docs.opencv.org/3.4.1/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981) k získání masky referenčního obdélníku.**

Funkce kromě zdrojového obrázku přijímá dolní a horní mez hodnoty pixelů ve formátu BGR (pozor celé OpenCV funguje v režimu BGR, ne RGB). Tedy dvě trojice (tuple) ve formátu (b, g, r), kde 0 <= b, g, r <= 255. Pixely s hodnotou >= než dolní mez a <= než horní mez budou mít ve výsledné masce hodnotu 255, ostatní 0. 

Dolní a horní mez naleznete experimentálně.

In [ ]:
ref_mask = cv2.inRange(img_patterns, ...) ###
show_images(ref_mask)

**3) Naleznete konturu která pokrývá nevětší plochu, otestujte aproximace kontury na obdélník.**

Výsledná maska by měla ideálně obsahovat pouze referenční obdelník, v obrazu se ale obecně může vyskytovat i šum. 
Nyní je nutné získat pozici referenčního obdelníku v obraze masky. 
Nejjednoduším způsobem je vyhledání kontur (obrysů) v obraze pomocí [cv2.findContours()](https://docs.opencv.org/3.3.1/d3/dc0/group__imgproc__shape.html#ga17ed9f5d79ae97bd4c7cf18403e1689a). Vyhledání kontur funguje jen na binárním černobílém obraze, proto jsme nejdříve museli využít cv2.inRange() k získání masky.

Konturu si lze představit jako křivku spojující několik bodů kolem obrysu souvislého objektu. Funkce cv2.findContours() příjímá navíc dva parametry, contour retrieval mode a contour approximation method. Těmito parametry se ve většině případů nemusíme zabývat. Zjednodušeně je tedy nastavíme `mode=cv2.RETR_LIST` a `method=cv2.CHAIN_APPROX_SIMPLE`. Pokud by vás přeci jen zajímaly, můžete si o nich přečíst [zde](https://docs.opencv.org/3.3.1/d4/d73/tutorial_py_contours_begin.html)

Vzhledem k šumu, který se na každém snímku vyskytuje, prakticky nikdy nenajdeme konturu pouze jednu. Je tedy nutné následně provést filtrování. V našem případě si vystačíme s výběrem kontury, která má největší plochu.

Posledním krokem je validace našeho postupu a vizualizace nalezené kontury pomocí funkce [cv2.drawContours()](https://docs.opencv.org/3.3.1/d6/d6e/group__imgproc__draw.html#ga746c0625f1781f1ffc9056259103edbc).
                                                                                

In [ ]:
_, contours, _  = cv2.findContours(...) ###
print(f'Found {len(contours)} contours.')

# pick only the contour with the biggest area
contour_biggest = max(contours, key=cv2.contourArea)
print(f'Biggest contour area: {cv2.contourArea(contour_biggest)}, coordinates:\n {contour_biggest.reshape((-1, 2)).tolist()}')
contour_drawn = cv2.drawContours(img_patterns.copy(), contours, -1, color=(255, 0, 0), thickness=5)
show_images(contour_drawn)

Z výsledných souřadnic vidíme, že kontura netvoří přesný obdelník, proto ji musíme obdelníkem aproximovat. To lze pomocí funkce [cv2.minAreaRect](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#ga3d476a3417130ae5154aea421ca7ead9). Návratovou hodnotou této funkce je tuple - (střed obdélníku (x,y), (výška, šířka), úhel rotace obdélníku). Jedna otázka zní, co je výška a co je šířka? Musíte to vždycky kontrolovat okem.

Pomocná funkce [cv2.boxPoints()](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#gaf78d467e024b4d7936cf9397185d2f5c) převádí nalezený obdelník z formátu (střed obdélníku (x,y), (šířka, výška), úhel rotace obdélníku) na 4 rohové body obdélníku ve formátu (x, y). To se může hodit např. k vizualizaci (lze použít jako vstup pro cv2.drawContours()). Pozor, pořadí vrácených bodů není zaručeno.

In [ ]:
rect = cv2.minAreaRect(contour_biggest) ###
print(f'Rect tuple: {rect}')
print(f'(cx, cy)={rect[0]}, (width, height)={rect[1]}, angle={rect[2]}')
print(f'Rect points: {cv2.boxPoints(rect).tolist()}')

**5) Nalezenou šířku referenčního obdelníku v pixelech můžeme konečně využít k získání poměru skutečné šířky obdélníku v reálných jednotkách a pixelové šířky obdélníku v obraze. Tento poměr budeme následně potřebovat k výpočtu rozměrů ostatních neznámých objektů.**

In [ ]:
ref_width_image, ref_height_image = ... ###
real_image_ratio = ... ### 
print(f'Ratio between real width and image width: {real_image_ratio}')

#### Pomocné funkce k další části

In [ ]:
def copy_to(src, dst, mask):
    '''Python alternative to C++/Java OpenCV's Mat.copyTo().
    More: https://docs.opencv.org/trunk/d3/d63/classcv_1_1Mat.html#a626fe5f96d02525e2604d2ad46dd574f'''
    locs = np.where(mask != 0) # Get the non-zero mask locations
    dst[locs[0], locs[1]] = src[locs[0], locs[1]]
    return dst

def midpoint(ptA, ptB):
    '''Returns the midpoint between two input points.'''
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

def order_points(pts):
    '''Sorts the points based on their x-coordinates.'''
    xSorted = pts[np.argsort(pts[:, 0]), :]

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (bl, tl) = leftMost

    # now that we have the top-left coordinate, use it as an
    # anchor to calculate the Euclidean distance between the
    # top-left and right-most points; by the Pythagorean
    # theorem, the point with the largest distance will be
    # our bottom-right point
    rightMost = rightMost[np.argsort(rightMost[:, 1]), :]
    (br, tr) = rightMost

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    return np.array([tl, tr, br, bl], dtype="float32")

def rotateImage(image, angle, image_center=None):
    """ Rotates the input image by specified angle.
    
    Parameters
    ----------
    image : np.ndarray
        Image to be rotated.
    angle : float
        Rotation angle.
    image_center : Optional[tuple(int, int)]
        Center of rotation.
    Returns
    -------
    np.ndarray
        Returns the rotated input image by specified angle.
    """
    if image_center is None:
        image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

# TODO: Most inefficient function in my whole life 
def draw_rotated_text(img, text, point, angle, text_scale, text_color, text_thickness):
    img_filled = np.full(img.shape, text_color, dtype=np.uint8)
    # create rotated text mask
    text_mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    cv2.putText(text_mask, "{:.2f} cm".format(text), point, 0, text_scale, (255, 255, 255), text_thickness)
    if angle > 0:
        angle = -angle + 90
    elif angle < 0:
        angle = angle + 90
    text_mask = rotateImage(text_mask, -angle, point)
    result = copy_to(img_filled, img.copy(), text_mask)
    return result


def draw_real_sizes(img, rect, width_text, height_text, lbl_size_scale=2, lbl_color=(0, 0, 255), lbl_thickness=8):
    tl, tr, br, bl = order_points(cv2.boxPoints(rect))
    mid_pt_width = midpoint(tl, tr)
    mid_pt_height = midpoint(tr, br)
    
    # bottom-left points where labels are drawn
    pt_label_first =  (int(mid_pt_width[0] - 10), int(mid_pt_width[1] - 10))
    pt_label_second = (int(mid_pt_height[0] + 10), int(mid_pt_height[1]))
        
    result = draw_rotated_text(img, width_text, pt_label_first, rect[2], lbl_size_scale, lbl_color, lbl_thickness)
    result = draw_rotated_text(result, height_text, pt_label_second, rect[2], lbl_size_scale, lbl_color, lbl_thickness)
    return result

### Úkol 2

Úkol 1 máme za sebou a už víme poměr mezi rozměrem v cm a pixelech, tím pádem už můžeme naměřit rozměry ostatních objektů na obrázku s neznámými rozměry.

**1) Opět využijte funkci [cv2.inRange()](https://docs.opencv.org/3.4.1/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981), tentokrát k získání masky objektů s neznámými skutečnými rozměry.**

**2) Nalezněte v nové masce snímku kontury stejně jako v případě referenčního objektu.**

**3) Následně je provedeno filtrování kontur podle jejich obsahu. Prahovou hodnotu obsahu v pixelech (threshold) je nutno zvolit experimentálně.**

In [ ]:
patterns_mask = ... ###
show_images(patterns_mask)

In [ ]:
_, contours, _  = ...
print(f'Found {len(contours)} contours.')

# Filter out noise
threshold = ... ###
contours =  [c for c in contours if cv2.contourArea(c) > threshold]
print(f'After filtering, {len(contours)} contours remained.')

contour_drawn = cv2.drawContours(img_patterns.copy(), contours, -1, color=(255, 0, 0 ), thickness=10)
show_images(contour_drawn)

# Sort contours by area. Just for better debugging.
contours.sort(key=cv2.contourArea, reverse=True)

**4) Nalezněte skutečné rozměry objektu**

Nyní potřebujeme pro jednotlivé kontury zjistit jejich skutečné rozměry, které chceme vizualizovat do výsledného obrázku.

Pro každou konturu tedy získáme jeji obdélníkovou aproximaci pomocí [cv2.minAreaRect](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#ga3d476a3417130ae5154aea421ca7ead9). Následně můžeme vypočítat skutečnou šířku a výšku objektu, díky předešlému vypočítanému poměru mezi skutečnou šířkou a pixelovou šířkou. Posledním krokem je volání funkce `draw_real_sizes()`, která se do vstupního obrázku pokusí vykreslit rozměry nalezeného objektu.

In [ ]:
# create a copy of original image
sizes_drawn = img_patterns.copy()

for c in contours:
    rect = ... ###
    shape_width, shape_height = ... ### 
    real_width = ...  ###
    real_height = ... ###
    
    cv2.drawContours(sizes_drawn, [c], -1, color=(255, 0, 0 ), thickness=5)
    sizes_drawn = draw_real_sizes(sizes_drawn, rect, real_width, real_height, 
                                  lbl_size_scale=.7, lbl_color=(0, 0, 255), lbl_thickness=1) ###

show_images(sizes_drawn)